### Creating swap routes 

We are going to try to create the swap routes based on the swaps that we get out of the solver. The idea is to figure out what the path is of every token in this network from the set of swaps. One thing we have to watch out for is if things are cycling or something. But we will check. 

1. Set up the solver and stuff so that we can get the outputs
2. Go through the output swaps and create the paths . 

In [1]:
import pandas as pd 

from tqdm import tqdm
from test_stephon import (
    OrderRoutingSimulationEnvironment, 
    Swap, 
    solve_with_known_eta, solve_with_unknown_eta, 
    get_tokens_from_trade
) 


In [75]:
chains: dict[str, list[str]] = {
"ETHEREUM": ["WETH", "USDC", "SHIBA"],
'CENTAURI': [],
"OSMOSIS": ["ATOM","SCRT"],
}

sim_env = OrderRoutingSimulationEnvironment(
    center_node='CENTAURI', 
    max_reserve=1e10, 
    chains=chains
)

# Solving with unknnown eta values
origin_token = "WETH"
number_of_init_tokens = 2000 
obj_token = "ATOM"


deltas, lambdas, psi, eta = solve_with_unknown_eta(
    origin_token, 
    number_of_init_tokens, 
    obj_token, 
    sim_env.all_tokens, 
    sim_env.all_cfmms, 
    sim_env.mapping_matrices
)

# We go through various values of eta and solve the problem for each of them
t_values = sorted([eta[i].value for i in range(len(sim_env.all_cfmms))])    
results = {} 

for j in tqdm(range(len(t_values))):
    example_eta = [int(eta[i].value >= t_values[j]) for i in range(len(sim_env.all_cfmms))]

    try: 
        deltas, lambdas, psi, objective = solve_with_known_eta(
            origin_token, 
            number_of_init_tokens, 
            obj_token, 
            sim_env.all_tokens, 
            sim_env.all_cfmms, 
            sim_env.mapping_matrices,
            example_eta
        )
        
        results[j] = {
            'deltas': [delta.value for delta in deltas],
            'lambdas': [lambda_.value for lambda_ in lambdas],
            'psi': psi.value,
            'objective': objective.value, 
            'eta': example_eta,
        }
        
    except:
        print(f"Failed for t={t_values[j]}")
        continue
    
# Getting the best result by looking at the objective value
best_result_key = max(results, key=lambda x: results[x]['objective'])
best_result = results[best_result_key]

# Taking the best result and getting the trades
best_eta = best_result['eta']
best_deltas = best_result['deltas']
best_lambdas = best_result['lambdas']
best_psi = best_result['psi']

chosen_cfmms = [cfmm for i, cfmm in enumerate(sim_env.all_cfmms) if best_eta[i] == 1]
chosen_mapping_matrices = [mapping_matrix for i, mapping_matrix in enumerate(sim_env.mapping_matrices) if best_eta[i] == 1]
chosen_deltas = [delta for i, delta in enumerate(best_deltas) if best_eta[i] == 1]
chosen_lambdas = [lambda_ for i, lambda_ in enumerate(best_lambdas) if best_eta[i] == 1]

# Creating the swaps from each of these
swaps = [] 

for i in range(len(chosen_cfmms)): 
    c = chosen_cfmms[i]
    m = chosen_mapping_matrices[i]
    d = chosen_deltas[i]
    l = chosen_lambdas[i]
    
    trade = m @ (l - d)
    
    tokens_from_trade = get_tokens_from_trade(trade, sim_env.all_tokens)
    
    if tokens_from_trade[c.tokens[0]] > 0:
        # This means that I got this token out from the CFMM
        out_token = c.tokens[0]
        in_token = c.tokens[1]
    else: 
        out_token = c.tokens[1]
        in_token = c.tokens[0]
    
    if c.is_transfer: 
        swap_type = 'transfer'
    else: 
        swap_type = 'swap'
        
    swaps.append(Swap(in_token, abs(tokens_from_trade[in_token]), out_token, abs(tokens_from_trade[out_token]), swap_type))

                                     CVXPY                                     
                                     v1.4.1                                    
(CVXPY) Jan 31 06:36:17 PM: Your problem has 200 variables, 91 constraints, and 0 parameters.
(CVXPY) Jan 31 06:36:17 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 31 06:36:17 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 31 06:36:17 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jan 31 06:36:17 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jan 31 06:36:17 PM: Compiling problem (target solver=CLARABEL)

/home/stephonh/.local/lib/python3.10/site-packages/cvxpy/problems/problem.py:1403: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
 78%|███████▊  | 31/40 [00:08<00:02,  3.61it/s]

Failed for t=1.1479206377689625e-07


 80%|████████  | 32/40 [00:08<00:02,  3.56it/s]

Failed for t=1.2906990992737703e-07


...

Failed for t=3.778618414783493e-07


 98%|█████████▊| 39/40 [00:10<00:00,  3.53it/s]

Failed for t=4.82867985511797e-07


100%|██████████| 40/40 [00:10<00:00,  3.78it/s]

Failed for t=0.02903686110160071


My idea for creating a route is to start at the origin token and keep a ledger of what is going on.
* Start with the start token and figure out where the swaps are where that is the in token
* Go through each of the out tokens and figure out where those are going into
* Keep going until there are no swaps left. 

In [76]:
swaps

[Swap(WETH, 1.733677580142127e-05, SHIBA, 2.2075150244578153e-06, swap),
 Swap(WETH, 717.4470088986723, CENTAURI/OSMOSIS/ATOM, 684.4736907544932, swap),
 Swap(WETH, 271.25608387565956, CENTAURI/OSMOSIS/SCRT, 269.9314752518846, swap),
 Swap(CENTAURI/OSMOSIS/SCRT, 269.9314741323891, CENTAURI/OSMOSIS/ATOM, 254.6989970479676, swap),
 Swap(ETHEREUM/WETH, 358.8729711085021, ETHEREUM/SHIBA, 345.3315477050431, swap),
 Swap(ETHEREUM/WETH, 627.9299144295995, OSMOSIS/ATOM, 589.8171779229936, swap),
 Swap(ETHEREUM/SHIBA, 345.33150550795074, OSMOSIS/ATOM, 340.35866885975435, swap),
 Swap(ETHEREUM/SHIBA, 4.147506096729254e-05, OSMOSIS/SCRT, 3.0363866243681217e-05, swap),
 Swap(SCRT, 1.3118478071234895e-05, CENTAURI/ETHEREUM/USDC, 8.174513881165166e-07, swap),
 Swap(WETH, 1011.2968887532132, ETHEREUM/WETH, 986.8028865721357, transfer),
 Swap(ETHEREUM/USDC, 1.6714490441154949e-06, USDC, 1.2637392694236498e-06, transfer),
 Swap(CENTAURI/OSMOSIS/ATOM, 939.1726866380393, OSMOSIS/ATOM, 911.0145935252402, 

In [77]:
def find_routes(swaps, start_token, end_token, current_route=None, visited=None):
    if current_route is None:
        current_route = []
    if visited is None:
        visited = set()

    current_route.append(start_token)
    visited.add(start_token)

    if start_token == end_token:
        return [current_route]

    routes = []
    for swap in swaps:
        if swap.in_token == start_token and swap.out_token not in visited:
            new_routes = find_routes(swaps, swap.out_token, end_token, current_route[:], visited.copy())
            routes.extend(new_routes)

    return routes
            


In [78]:
start_token = 'WETH'
end_token = 'ATOM'

routes = find_routes(swaps, start_token, end_token)

In [79]:
def filter_swaps_by_in_and_out(swaps, in_token, out_token):
    return [swap for swap in swaps if swap.in_token == in_token and swap.out_token == out_token]

In [80]:
for route in routes: 
    print(route)

['WETH', 'CENTAURI/OSMOSIS/ATOM', 'OSMOSIS/ATOM', 'ATOM']
['WETH', 'CENTAURI/OSMOSIS/SCRT', 'CENTAURI/OSMOSIS/ATOM', 'OSMOSIS/ATOM', 'ATOM']
['WETH', 'ETHEREUM/WETH', 'ETHEREUM/SHIBA', 'OSMOSIS/ATOM', 'ATOM']
['WETH', 'ETHEREUM/WETH', 'OSMOSIS/ATOM', 'ATOM']


In [81]:
# Getting the max length of the routes
max_length = max([len(route) for route in routes])

In [82]:
levels = {}
for i in range(1, max_length):
    roots = set([tuple(route[i-1:i+1]) for route in routes if len(route) > i]) 
    levels[i] = roots
    

In [92]:
levels

{1: {('WETH', 'CENTAURI/OSMOSIS/ATOM'),
  ('WETH', 'CENTAURI/OSMOSIS/SCRT'),
  ('WETH', 'ETHEREUM/WETH')},
 2: {('CENTAURI/OSMOSIS/ATOM', 'OSMOSIS/ATOM'),
  ('CENTAURI/OSMOSIS/SCRT', 'CENTAURI/OSMOSIS/ATOM'),
  ('ETHEREUM/WETH', 'ETHEREUM/SHIBA'),
  ('ETHEREUM/WETH', 'OSMOSIS/ATOM')},
 3: {('CENTAURI/OSMOSIS/ATOM', 'OSMOSIS/ATOM'),
  ('ETHEREUM/SHIBA', 'OSMOSIS/ATOM'),
  ('OSMOSIS/ATOM', 'ATOM')},
 4: {('OSMOSIS/ATOM', 'ATOM')}}

In [94]:
for level, roots in levels.items(): 
    for s in roots: 
        print(s)

('WETH', 'ETHEREUM/WETH')
('WETH', 'CENTAURI/OSMOSIS/SCRT')
('WETH', 'CENTAURI/OSMOSIS/ATOM')
('CENTAURI/OSMOSIS/SCRT', 'CENTAURI/OSMOSIS/ATOM')
('ETHEREUM/WETH', 'OSMOSIS/ATOM')
('ETHEREUM/WETH', 'ETHEREUM/SHIBA')
('CENTAURI/OSMOSIS/ATOM', 'OSMOSIS/ATOM')
('ETHEREUM/SHIBA', 'OSMOSIS/ATOM')
('OSMOSIS/ATOM', 'ATOM')
('CENTAURI/OSMOSIS/ATOM', 'OSMOSIS/ATOM')
('OSMOSIS/ATOM', 'ATOM')


In [96]:
for level, roots in levels.items():
    swaps_at_level = []
    for s in roots: 
        in_token, out_token = s
        filtered_swaps = filter_swaps_by_in_and_out(swaps, in_token, out_token)
        swaps_at_level.extend(filtered_swaps)

    print(f'For level {level} we have swaps:\n')
    for swap in swaps_at_level: 
        print(swap)
    print('\n\n')

For level 1 we have swaps:

Swap(WETH, 1011.2968887532132, ETHEREUM/WETH, 986.8028865721357, transfer)
Swap(WETH, 271.25608387565956, CENTAURI/OSMOSIS/SCRT, 269.9314752518846, swap)
Swap(WETH, 717.4470088986723, CENTAURI/OSMOSIS/ATOM, 684.4736907544932, swap)



For level 2 we have swaps:

Swap(CENTAURI/OSMOSIS/SCRT, 269.9314741323891, CENTAURI/OSMOSIS/ATOM, 254.6989970479676, swap)
Swap(ETHEREUM/WETH, 627.9299144295995, OSMOSIS/ATOM, 589.8171779229936, swap)
Swap(ETHEREUM/WETH, 358.8729711085021, ETHEREUM/SHIBA, 345.3315477050431, swap)
Swap(CENTAURI/OSMOSIS/ATOM, 939.1726866380393, OSMOSIS/ATOM, 911.0145935252402, transfer)



For level 3 we have swaps:

Swap(ETHEREUM/SHIBA, 345.33150550795074, OSMOSIS/ATOM, 340.35866885975435, swap)
Swap(OSMOSIS/ATOM, 1841.1904396677062, ATOM, 1794.7325516191295, transfer)
Swap(CENTAURI/OSMOSIS/ATOM, 939.1726866380393, OSMOSIS/ATOM, 911.0145935252402, transfer)



For level 4 we have swaps:

Swap(OSMOSIS/ATOM, 1841.1904396677062, ATOM, 1794.73255161

In [85]:
levels

{1: {('WETH', 'CENTAURI/OSMOSIS/ATOM'),
  ('WETH', 'CENTAURI/OSMOSIS/SCRT'),
  ('WETH', 'ETHEREUM/WETH')},
 2: {('CENTAURI/OSMOSIS/ATOM', 'OSMOSIS/ATOM'),
  ('CENTAURI/OSMOSIS/SCRT', 'CENTAURI/OSMOSIS/ATOM'),
  ('ETHEREUM/WETH', 'ETHEREUM/SHIBA'),
  ('ETHEREUM/WETH', 'OSMOSIS/ATOM')},
 3: {('CENTAURI/OSMOSIS/ATOM', 'OSMOSIS/ATOM'),
  ('ETHEREUM/SHIBA', 'OSMOSIS/ATOM'),
  ('OSMOSIS/ATOM', 'ATOM')},
 4: {('OSMOSIS/ATOM', 'ATOM')}}

In [87]:
swaps

[Swap(WETH, 1.733677580142127e-05, SHIBA, 2.2075150244578153e-06, swap),
 Swap(WETH, 717.4470088986723, CENTAURI/OSMOSIS/ATOM, 684.4736907544932, swap),
 Swap(WETH, 271.25608387565956, CENTAURI/OSMOSIS/SCRT, 269.9314752518846, swap),
 Swap(CENTAURI/OSMOSIS/SCRT, 269.9314741323891, CENTAURI/OSMOSIS/ATOM, 254.6989970479676, swap),
 Swap(ETHEREUM/WETH, 358.8729711085021, ETHEREUM/SHIBA, 345.3315477050431, swap),
 Swap(ETHEREUM/WETH, 627.9299144295995, OSMOSIS/ATOM, 589.8171779229936, swap),
 Swap(ETHEREUM/SHIBA, 345.33150550795074, OSMOSIS/ATOM, 340.35866885975435, swap),
 Swap(ETHEREUM/SHIBA, 4.147506096729254e-05, OSMOSIS/SCRT, 3.0363866243681217e-05, swap),
 Swap(SCRT, 1.3118478071234895e-05, CENTAURI/ETHEREUM/USDC, 8.174513881165166e-07, swap),
 Swap(WETH, 1011.2968887532132, ETHEREUM/WETH, 986.8028865721357, transfer),
 Swap(ETHEREUM/USDC, 1.6714490441154949e-06, USDC, 1.2637392694236498e-06, transfer),
 Swap(CENTAURI/OSMOSIS/ATOM, 939.1726866380393, OSMOSIS/ATOM, 911.0145935252402, 